In [2]:
from quantfreedom.backtester.nb.buy_funcs import long_increase_nb
from quantfreedom.backtester.enums.enums import *


In [ ]:
account_state = AccountState(
    available_balance=1000.,
    equity=1000.
)
entry_order = EntryOrder(
    max_equity_risk_pct=1.
    risk_rewards=2,
    size_pct=.01,
    sl_pcts=.01
)
order_results = OrderResult(
    
)
static_variables = StaticVariables(
    
)

In [ ]:
long_increase_nb(
    
)